In [ ]:
import os
import csv
from datetime import date, datetime
import yaml
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from experiment.toolkits.plotting_helper import *

project_dir = Path().resolve().parents[1]
sys.path.insert(0, str(project_dir))
from time import sleep
import pickle
import datetime
from scipy.stats import linregress

In [ ]:
os.chdir(str(project_dir))

In [ ]:
base_cmd = "./bin/modules/image-processing-benchmarking/filtering-test"
def get_processing_time (N_t, kernel_size, reps):
    psf_bin_path = str(project_dir) + "/resources/psf/psf_" + str(N_t) + '_' + str(kernel_size) + ".bin"
    if not os.path.isfile(psf_bin_path):
        psf_pickle_path = str(project_dir) + "/resources/psf/psf_" + str(N_t) + '_' + str(kernel_size) + ".pickle"
        os.popen("python " + str(project_dir) + "/tools/generate_fake_psf.py" + " " + str(N_t) + " " + str(kernel_size) + " " + psf_pickle_path)
        os.popen(str(project_dir) + "/tools/psf-translator.py --pickle " + psf_pickle_path + " --address " + psf_bin_path)
    command = base_cmd + " " + psf_bin_path + " " + str(N_t) + ' ' + str(kernel_size) + ' ' + str(reps) 
    return os.popen(command).read()

In [ ]:
reps = 1000
N_t = [8,16,32,64,128,256,512,1024,2048] 
kernel_size = [1, 3, 5, 7, 9]
results = [] 

for i in range(len(kernel_size)):
    print(f"{i}/{len(kernel_size)}")

    ks = kernel_size[i]
    temp = []
    for nt in N_t:
        acq_time_str = get_processing_time(nt, ks, reps)
        temp.append(tuple(acq_time_str.split('\n')[0], acq_time_str.split('\n')[1]))
    results.append(temp)

# Save results

In [ ]:
# Saving the results 
file_name = datetime.datetime.now().strftime("%Y-%m-%d, %H-%M-%S.pickle")
with open(os.path.join(str(project_dir), "resources", "image-processing-characterization", file_name), "wb") as file:
    pickle.dump(
        {
            "xvals": N_t,
            "kernels": kernel_size,
            "experiment": file_name,
            "data": results 
        }, file
        )

# Load results

In [ ]:
load_latest = True
if not load_latest:
    file_name = "2024-04-22, 19-04-01.pickle"

with open(os.path.join(str(project_dir), "resources", "image-processing-characterization", file_name), "rb") as file:
    data = pickle.load(file)
    N_t = data["xvals"]
    kernel_size = data["kernels"]
    title = data["experiment"]
    results = data["data"]

In [ ]:
acq_time_mean = time_data.mean(axis = 1) / 1e6
thr_acquisition_time = np.array([theoretical_acquistion_time_oct3(H=h, flag_dma=False)*1000 for h in roi_heights])
acq_time_std = time_data.std(axis = 1)

xvals = roi_heights / 1024 * 2
yvals = acq_time_mean - thr_acquisition_time
result = linregress(xvals, yvals)

fig, ax = getStylishFigureAxes(1, 1, axes_list=True, dpi=300)
ErrorbarPlotter(
    fig,
    ax[0],
    xvals,
    yvals,
    yerr = acq_time_std / 1e6,
    style = {"marker": "*", "linestyle": "None", "markersize": 1},
    xlim = [-0.1, 2.1],
    xticks = [0, 1, 2],
    xlabel = "Image size (MB)",
    ylabel = "$t_{transfer}$ (ms)",
    ylim = [-0.5, 10.5],
    yticks = [0, 5, 10],
    title = title.split(".")[0]
).draw()

PlotPlotter(
    fig,
    ax[0],
    xvals,
    xvals * result.slope + result.intercept,
    style = {"linestyle" : "--", "color": "blue", "lw": 1,
             "label": "$f_{PCIe}$ = " + f"{str(round(1 / result.slope, 2))} GB/s"}
).draw()

ax[0].legend(fontsize=6)
plt.savefig("result.jpg")
plt.show()